<a href="https://colab.research.google.com/github/wooookim/ML-DL/blob/main/%EC%84%9C%EC%9A%B8%EC%8B%9C_%ED%8F%89%EA%B7%A0_%EA%B8%B0%EC%98%A8_%EC%98%88%EC%B8%A1_13.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
uploaded = files.upload()

Saving train.csv to train.csv


In [2]:
import pandas as pd
data = pd.read_csv('/content/train.csv')
submission = pd.read_csv('/content/sample_submission.csv')

In [3]:
# '일시' 열을 날짜 데이터로 변환
data['일시'] = pd.to_datetime(data['일시'])

# '일시' 컬럼에서 '월', '일' 열 추가
data['월'] = data['일시'].dt.month
data['일'] = data['일시'].dt.day

# 월, 일별 중앙값 계산
monthly_medians = data.groupby('일').transform('median')
daily_medians = data.groupby('일').transform('median')

<ipython-input-3-f0d9f2a84bd0>:9: FutureWarning: The default value of numeric_only in DataFrameGroupBy.median is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  monthly_medians = data.groupby('일').transform('median')
<ipython-input-3-f0d9f2a84bd0>:10: FutureWarning: The default value of numeric_only in DataFrameGroupBy.median is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  daily_medians = data.groupby('일').transform('median')


In [13]:
from datetime import datetime

current_year = datetime.now().year

# 2월 29일에 해당하는 행 선택
selected_rows = data[(data['일시'].dt.month == 2) & (data['일시'].dt.day == 29)]

# 29일에 해당하는 16개 행
# 23011 - 16 = 22995

# 선택한 행의 인덱스를 사용하여 삭제
data = data.drop(selected_rows.index)

print(data.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22995 entries, 0 to 23010
Data columns (total 13 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   일시      22995 non-null  datetime64[ns]
 1   최고기온    22992 non-null  float64       
 2   최저기온    22992 non-null  float64       
 3   일교차     22991 non-null  float64       
 4   강수량     9142 non-null   float64       
 5   평균습도    22995 non-null  float64       
 6   평균풍속    22991 non-null  float64       
 7   일조합     22877 non-null  float64       
 8   일사합     18137 non-null  float64       
 9   일조율     22630 non-null  float64       
 10  평균기온    22995 non-null  float64       
 11  월       22995 non-null  int64         
 12  일       22995 non-null  int64         
dtypes: datetime64[ns](1), float64(10), int64(2)
memory usage: 2.5 MB
None


윤년 데이터 빼고 365로 나눠서 모델에 적용시키기

In [14]:
train = data[['최고기온', '최저기온', '일교차', '강수량', '평균습도', '평균풍속', '일조합', '일사합', '일조율']]

target = data['평균기온']

In [20]:
# 결측값 대체
data.fillna(daily_medians, inplace=True)

from xgboost import XGBRegressor
xgb = XGBRegressor()
xgb.fit(train, target)

pred_xgb = xgb.predict(train)

from sklearn.metrics import mean_squared_error
xgb_mae = mean_squared_error(target, pred_xgb)
print(xgb_mae)

submission['평균기온'] = pred_xgb[:358]
print(submission.head(10))

0.14312530580469168
         일시      평균기온
0 -2.011936 -2.011936
1 -1.901870 -1.901870
2  3.772155  3.772155
3  6.986705  6.986705
4 -4.083561 -4.083561
5 -5.160858 -5.160858
6  1.405020  1.405020
7  1.828838  1.828838
8  2.475737  2.475737
9  1.886170  1.886170


In [19]:
# 월 결측값 대체
data.fillna(monthly_medians, inplace=True)

from xgboost import XGBRegressor
xgb = XGBRegressor()
xgb.fit(train, target)

pred_xgb = xgb.predict(train)

from sklearn.metrics import mean_squared_error
xgb_mae = mean_squared_error(target, pred_xgb)
print(xgb_mae)

submission['평균기온'] = pred_xgb[:358]
print(submission.head(10))

0.14312530580469168
         일시      평균기온
0 -2.011936 -2.011936
1 -1.901870 -1.901870
2  3.772155  3.772155
3  6.986705  6.986705
4 -4.083561 -4.083561
5 -5.160858 -5.160858
6  1.405020  1.405020
7  1.828838  1.828838
8  2.475737  2.475737
9  1.886170  1.886170


타겟이 정렬된 형태라서 똑같이 나오나...?

스케일링 적용해서 결과 확인해보기

In [22]:
# 결측값 대체
data.fillna(daily_medians, inplace=True)

from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
ss.fit(train)
train_scaled = ss.transform(train)

from xgboost import XGBRegressor
xgb = XGBRegressor()
xgb.fit(train_scaled, target)

pred_xgb = xgb.predict(train_scaled)

from sklearn.metrics import mean_squared_error
xgb_mae = mean_squared_error(target, pred_xgb)
print(xgb_mae)

submission['평균기온'] = pred_xgb[:358]
print(submission.head(10))

0.14312530580469168
         일시      평균기온
0 -2.011936 -2.011936
1 -1.901870 -1.901870
2  3.772155  3.772155
3  6.986705  6.986705
4 -4.083561 -4.083561
5 -5.160858 -5.160858
6  1.405020  1.405020
7  1.828838  1.828838
8  2.475737  2.475737
9  1.886170  1.886170


**스케일링 전과 후가 동일한 결과**

스케일링의 효과가 적은 경우: XGBoost는 트리 기반 모델로, 트리 알고리즘은 입력 특성의 스케일에 민감하지 않을 수 있어. 따라서 특히 스케일링이 큰 영향을 주지 않을 경우 결과가 거의 같을 수 있어.

과적합의 가능성: 모델이 학습 데이터에 너무 맞춰져서 과적합되었을 가능성이 있어. 학습 데이터에 대해 성능이 좋아질 수 있지만, 검증 데이터나 테스트 데이터에 대한 성능이 떨어질 수 있어.

하이퍼파라미터 튜닝의 필요성: 모델의 성능을 향상시키기 위해 XGBoost의 하이퍼파라미터를 튜닝할 필요가 있을 수 있어. 특히 학습률, 트리 깊이, 트리 개수 등을 조절하는 것이 도움이 될 수 있어.

스케일링 효과를 보기 위해서는 검증, 테스트 데이터를 분리할 필요가 있다